<h1>DECISION TREE ALGORITHAM ID-3</h1
>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


DATASET


In [ ]:
import numpy as np
import pandas as pd

df=pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Lab 4 ML Dataset - ID3.xlsx")

df.columns =df.iloc[1]
#print(df)
df1 =df.iloc[23:25]
replacement_dict = {0: 'yes', 1: 'no'}

df1.loc[23, 'Answer'] = 'yes'
df1.loc[24, 'Answer'] = 'no'
#for testing dataframe df1

#for training dataframe df2
df2 =df.iloc[2:16]

FUNCTION DECLARATION AREA


ALL THE FUNDAMENTAL FUNCTIONS WHICH IS REQUIRED TO BE DECLARE HERE

In [ ]:

def for_pos_neg_of_table( Answer,positive,negative):

  for i in Answer:
    if i=='yes':
      positive=positive+1
    else :
      negative=negative+1

  return positive,negative

def info(row,pos,neg):
  p =pos/row
  q=neg/row
  sum=0;
  if p!=0: sum +=-p * np.log2(p)
  if q!=0: sum+=- q * np.log2(q)

  return sum
#my_dict = {}

def entropy1( col,col_name,Answer,positive,negative,total_count,df4,dct):
      listed =list(df4[col_name])

      #print(col)
      for set_val in col:
         #print(set_val)
         for i in range(len(listed)):
                if set_val==listed[i] and Answer[i]=='yes':
                  positive=positive+1
                elif set_val==listed[i] and Answer[i]=='no':
                  negative=negative+1
         #print(positive,negative)
         dct[set_val] = (positive, negative)
         positive =0
         negative=0
      return dct

def calculate_pos_neg_each(df3,list_Answer,dict_my):
  #print(df3.columns)
  for each_col in df3.columns:
    if each_col=='Answer':
      continue

    set_col=set(df3[each_col])
    #print(set_col)
    s= entropy1(set_col,each_col,list_Answer,0,0,0,df3,dict_my)

  return dict_my
#info_dict ={}
def info_gain_column(df3,my_dict,total_num_of_row,info_dict):

  for each_col in df3.columns:
    if each_col=='Answer':
      continue

    set_col=set(df3[each_col])
    #print(set_col)
    n =len(set_col)
    i=0
    total=0
  # for i in range(i,n):
    p=0
    for val in set_col:

      pos=my_dict[val][0]
      neg=my_dict[val][1]
      row1=pos+neg
      p +=row1/total_num_of_row*(info(row1,pos,neg))

    info_dict[each_col]=p
  return info_dict

def give_the_node_frame(df3,max_key):
  df = pd.DataFrame(df3)
  l=[]
  l2=[]
  # Group the data by the 'age' column and store each group's DataFrame in a dictionary
  grouped_data = {}
  for age_group, group_data in df3.groupby(max_key):
      # Remove the 'age' column if needed
      group_data = group_data.drop(max_key, axis=1)
      grouped_data[age_group] = group_data
      l.append(age_group)
  # Access the DataFrames for each age group
  for i, val in enumerate(l):
    #print(val)
    l2.append(grouped_data[val])

  return l2

THE FUNCTION WHICH GIVE YOU DECISION NODE

In [ ]:

def give_the_node(df3):
  total_num_of_col =len(df3.columns)
  #print(total_num_of_col)

  total_num_of_row =0;
  for i in df3.iterrows():
   total_num_of_row=total_num_of_row+1

  #print(total_num_of_row)
  my_dict = {}
  list_Answer =list(df3['Answer'])
  positive,negative=for_pos_neg_of_table(list_Answer,0,0)

  info_gain =info(total_num_of_row,positive,negative)

  #print(info_gain)

  d=calculate_pos_neg_each(df3,list_Answer,my_dict)
  #print(d)
  info_dict1 ={}
  E =info_gain_column(df3,d,total_num_of_row,info_dict1)
  #print(E)

  max_key = max(E, key=lambda k: abs(info_gain-E[k]))

  return max_key


THIS FUNCTION GIVE THE NEXT LEVEL DECISION NODE AND CORRESPONDING IT ALL TABLES


In [ ]:
from collections import defaultdict
def give_the_chiled_node(node_frame):
  next_level_dict = defaultdict(list)
  for i, frame in enumerate(node_frame):
    node = give_the_node(frame)
    temp_list=give_the_node_frame(frame,node)
    next_level_dict[node].append(temp_list)

  return next_level_dict


THE FUNCTION WHICH PRINT DECISION DETAILS AND LIST ALL DATA FRAME INTO A PARTICULAR LIST


In [ ]:

def print_Decision_Node(take_as_dict,level_by):
    keys_list = list(take_as_dict.keys())
    #print(keys_list)
    for key_to_access in keys_list:
      i = -1
    # Assuming you have a list called 'chiled_node_at_next_level' and a key called 'key_to_access'
    #key_to_access = 'age'
      print("Dicision node At level ",level_by,"is :",key_to_access)
      #print("_______________________________________________________________________________")
    # Check if the key exists in the dictionary
      if key_to_access in take_as_dict:
          # Access the list of data frames associated with the key
          data_frames_list = take_as_dict[key_to_access]

          # Create an empty list to store values
          values = []

          # Iterate through the data frames and decrement i
          while data_frames_list and i>-4:
            for data_frame in data_frames_list:
                #print(data_frame)
                if abs(i) <= len(data_frame):
                    #print(i)
                    # Access the element at index -i in the data frame
                    value = data_frame[i]
                    data=pd.DataFrame(value)
                    print("---------------------------------------------------------------")
                    print(data)
                    print("---------------------------------------------------------------")

                    if all(data['Answer'] == 'yes'):
                      #print("Leaf Node yes \n",key_to_access)
                      continue
                    elif all(data['Answer'] == 'no'):
                      #print("Leaf Node no  \n",key_to_access)
                      continue
                    else:
                      #print("Values in the 'Answer' column are mixed \n")
                      pass

                    values.append(data)

                else: break

      else:
          print(f"Key '{key_to_access}' not found in the dictionary.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    return values

RECURSION CODE FOR TRAINING

In [ ]:
ls =[df2]
def Recursion_tree(dicision_list,level):
    chiled_node_at_next_level = give_the_chiled_node(dicision_list)
    if len(chiled_node_at_next_level) == 0:
        return  # Terminate recursion
    else:
        #print(chiled_node_at_next_level)
        chiled_node_list = print_Decision_Node(chiled_node_at_next_level,level)
        Recursion_tree(chiled_node_list,level+1)  # Continue recursion



Recursion_tree(ls,0)

Dicision node At level  0 is : age
---------------------------------------------------------------
1   income student credit_rating Answer
2     high      no          fair     no
3     high      no     excellent     no
9   medium      no          fair     no
10     low     yes          fair    yes
12  medium     yes     excellent    yes
---------------------------------------------------------------
---------------------------------------------------------------
1   income student credit_rating Answer
5   medium      no          fair    yes
6      low     yes          fair    yes
7      low     yes     excellent     no
11  medium     yes          fair    yes
15  medium      no     excellent     no
---------------------------------------------------------------
---------------------------------------------------------------
1   income student credit_rating Answer
4     high      no          fair    yes
8      low     yes     excellent    yes
13  medium      no     excellent    yes
14   

RECURSION CODE FOR TESTING

In [ ]:
l =[df1]
def Recursion_tree(dicision_list,level):
    chiled_node_at_next_level = give_the_chiled_node(dicision_list)
    if len(chiled_node_at_next_level) == 0:
        return  # Terminate recursion
    else:
        #print(chiled_node_at_next_level)
        chiled_node_list = print_Decision_Node(chiled_node_at_next_level,level)
        Recursion_tree(chiled_node_list,level+1)  # Continue recursion



Recursion_tree(l,0)

Dicision node At level  0 is : age
---------------------------------------------------------------
1  income student credit_rating Answer
23  high      yes     excellent    yes
---------------------------------------------------------------
---------------------------------------------------------------
1  income student credit_rating Answer
24    low      no     excellent     no
---------------------------------------------------------------
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
